# Описание задачи

## Производственная практика 2022/2023 в Университете ИТМО "Построение модели машинного обучения для предсказания спроса на товары"
### Мне в рамках индивидуального задания необходимо разработать модель машинного обучения, которая будет предсказывать спрос на товары, то есть решать задачу регрессии временных рядов
### Для этой задачи я взял самый популярный доступный [датасет](https://www.kaggle.com/competitions/demand-forecasting-kernels-only) по предсказанию спроса на товары

# Импорт библиотек

In [275]:
import random

import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

# Чтение датасета

In [213]:
df = pd.read_csv('data/train.csv', parse_dates=['date'])

In [214]:
df.sample(10)

,date,store,item,sales
464354,2014-07-05,5,26,60
363965,2014-08-15,10,20,58
386821,2017-03-16,2,22,92
216233,2015-02-05,9,12,53
413753,2015-12-14,7,23,13
330261,2017-05-01,1,19,43
874066,2016-05-23,9,48,57
753883,2017-04-21,3,42,46
138342,2016-10-24,6,8,57
187554,2016-07-26,3,11,93


В нашем датасете содержатся данные за 5 лет о покупках в 10 магазинах 50 товаров
Задание на kaggle подразумевает предсказание спроса на эти 50 товаров в 10 магазинах на следующие 3 месяца, однако в рамках практики мы облегчим задачу. Нашей задачей будет предсказать стоимость 1 товара в 1 магазине за 1 год (последний, 5-ый год)

# Исследование датасета

In [215]:
df.shape

(913000, 4)

In [216]:
df.isna().sum()

date     0
store    0
item     0
sales    0
dtype: int64

In [217]:
df.describe()

,store,item,sales
count,913000.000000,913000.000000,913000.000000
mean,5.500000,25.500000,52.250287
std,2.872283,14.430878,28.801144
min,1.000000,1.000000,0.000000
25%,3.000000,13.000000,30.000000
50%,5.500000,25.500000,47.000000
75%,8.000000,38.000000,70.000000
max,10.000000,50.000000,231.000000


In [218]:
df.dtypes

date     datetime64[ns]
store             int64
item              int64
sales             int64
dtype: object

Описание полей датасета:
- `date` - Дата
- `store` - Id магазина
- `item` - Id товара
- `sales` - Количество продаж текущего товара в текущем магазине на текущую дату

# Краткий анализ распределения переменных

In [219]:
all(df['date'].value_counts() == 10 * 50) # 10 магазинов * 50 товаров

True

In [220]:
all(df['store'].value_counts() == 50 * 1826) # 50 товаров * 5 лет

True

In [221]:
all(df['item'].value_counts() == 10 * 1826) # 10 товаров * 5 лет

True

## Немного графиков

In [222]:
px.line(df.groupby(['date'])[['sales']].sum(),
        title='Сумма продаж всех товаров за каждый день по всем магазинам')

In [223]:
px.line(df.groupby([pd.Grouper(freq='MS', key='date')])[['sales']].sum(),
        title='Сумма продаж всех товаров за каждый месяц по всем магазинам')

Распределение суммы продаж по товарам и магазинам

In [224]:
px.bar(pd.pivot_table(df, values='sales',
               columns='store',
               index='item',
               aggfunc='sum'), title="Распределение суммы продаж по товарам и магазинам",
       template='plotly_dark')

Распределение суммы продаж по товарам, магазинам и годам

In [225]:
df.groupby(['item', 'store' ,pd.Grouper(freq='Y', key='date')]).sum().unstack().unstack()

sales                                                          \
date  2013-12-31                                                           
store         1      2      3      4      5      6      7      8      9    
item                                                                       
1           6025   8530   7603   6973   5092   5101   4704   8122   7063   
2          16036  22942  20210  18898  13391  13590  12490  21711  18922   
3          10100  14165  12688  11710   8540   8556   7646  13625  11800   
4           6045   8590   7764   7076   5011   5139   4700   8323   7110   
5           5002   7068   6270   5882   4329   4205   3796   6832   5916   
6          16182  22989  20149  18671  13594  13630  12385  21784  18626   
7          16293  22987  20188  18885  13453  13474  12301  21981  18802   
8          21112  30233  26222  24680  17779  17848  16464  28457  24446   
9          14201  19816  17849  16541  11788  12034  10985  19027  16461   
10         19988  28604  25189  23278  16842  16905  15628  27136  23329   
11         19369  27236  24253  22309  16244  16084  14511  26019  22285   
12         19299  27038  24280  22388  16131  16361  14912  26000  22212   
13         23057  33000  29189  27029  19721  19427  17829  31562  26765   
14         16086  22965  20211  18737  13732  13570  12679  21804  18561   
15         24101  34352  30285  28221  20419  20348  18378  32961  28304   
16          7073   9780   8944   8260   5914   5969   5373   9645   8216   
17          9286  13039  11413  10470   7675   7588   6897  12378  10675   
18         23181  33191  29299  26917  19671  19198  17872  31433  27001   
19         10997  15848  13951  12902   9321   9241   8453  15157  12902   
20         13101  18344  16591  15242  11120  10976  10008  17572  15249   
21         10905  15837  14033  12839   9211   9400   8567  15365  13031   
22         22011  31433  28026  25827  18871  18658  17069  30134  25607   
23          8008  11551  10155   9085   6704   6742   6131  10897   9409   
24         18320  25999  22682  21078  15285  15333  13971  24789  21187   
25         21932  31113  27893  25955  18423  18673  17097  30057  25767   
26         13034  18477  16381  15059  11089  10898   9992  17745  15317   
27          6133   8592   7590   7041   5128   5074   4608   8254   7009   
28         24237  34293  30420  28159  20167  20147  18403  32950  28241   
29         19102  27120  24323  22348  15897  16368  14792  25608  22075   
30         10842  15653  13962  13016   9331   9410   8480  15049  12933   
31         16134  22997  20311  18585  13392  13486  12378  21914  18832   
32         12088  17197  15201  14053   9892  10021   9339  16449  14217   
33         19243  27183  24154  22506  16004  15942  14907  25721  22000   
34          7085  10111   8851   8167   5986   5913   5331   9657   8113   
35         18001  25726  22934  20860  15261  15107  14029  24580  21275   
36         20795  29961  26536  24529  17723  17863  16348  28562  24923   
37          8081  11223   9975   9296   6897   6879   5999  10800   9376   
38         22540  31550  28012  25678  18667  18606  17078  30001  25674   
39         11995  17101  15184  13928  10022  10196   9385  16418  13873   
40          8110  11371  10189   9254   6660   6775   6193  10895   9321   
41          6151   8538   7456   7132   5068   4959   4742   8142   6984   
42          9966  14329  12641  11867   8456   8582   7647  13775  11886   
43         14116  20156  17681  16320  11958  11893  10791  19182  16542   
44          8056  11651  10204   9469   6807   6877   6196  11128   9415   
45         22337  31655  27954  25669  18633  18324  16885  30123  25615   
46         15968  23059  20327  18788  13451  13680  12357  21924  18824   
47          5989   8535   7575   6950   5118   5160   4697   8383   6955   
48         14059  20163  17737  16182  11981  11785  10793  19020  16369   
49          8003  11367  10176   9256   6794   6779   6127 

# Выберем (случайным образом) нашу целевую связку магазин - товар

In [83]:
# random_shop = random.randint(df['store'].min(), df['store'].max() + 1)
# random_item = random.randint(df['item'].min(), df['item'].max() + 1)

In [84]:
# print(f"Наша целевая связка: магазин {random_shop} + товар {random_item}")

Наша целевая связка: магазин 5 + товар 44


### Наша целевая связка: магазин 5 + товар 44

In [226]:
target_pair = (5, 44)

In [227]:
temp_df = df.groupby(['store', 'item'])[['sales']].sum().reset_index()
temp_df[(temp_df['item'] == 44) & (temp_df['store'] == 5)]

,store,item,sales
243,5,44,40930


In [228]:
print(f"Среднее: {temp_df['sales'].mean()}, СКО: {temp_df['sales'].std()}, Максимум: "
      f"{temp_df['sales'].max()}, Минимум: {temp_df['sales'].min()}")

Среднее: 95409.024, СКО: 43938.190666269766, Максимум: 205677, Минимум: 23252


# Получилось так, что у нас далеко не самая популярная связка товара и магазина, но всё равно попробуем предсказать именно её. Если быть точнее, то предскажем спрос на этот товар в этом магазине в 2017 году, а история продаж всех товаров во всех магазинах за 2013 - 2016 года будет нашей обучающей выборкой

# Feature Engineering

In [229]:
df = df.merge(df.groupby(['store',
                          'date'])[['sales']].sum().reset_index().rename(columns={'sales': 'store_sales'}),
              on=['store', 'date'], how='left')

In [230]:
df = df.merge(df.groupby(['item',
                          'date'])[['sales']].sum().reset_index().rename(columns={'sales': 'item_sales'}),
              on=['item', 'date'], how='left')

In [231]:
df['sales_lag_365_days'] = df.groupby(["store", "item"])['sales'].transform(lambda x: x.shift(365))
df['store_sales_lag_365_days'] = df.groupby(["store", "item"])['store_sales'].transform(lambda x: x.shift(365))
df['item_sales_lag_365_days'] = df.groupby(["store", "item"])['item_sales'].transform(lambda x: x.shift(365))

In [232]:
df

,date,store,item,sales,store_sales,item_sales,sales_lag_365_days,store_sales_lag_365_days,item_sales_lag_365_days
0,2013-01-01,1,1,13,1316,133,NaN,NaN,NaN
1,2013-01-02,1,1,11,1264,99,NaN,NaN,NaN
2,2013-01-03,1,1,14,1305,127,NaN,NaN,NaN
3,2013-01-04,1,1,13,1452,145,NaN,NaN,NaN
4,2013-01-05,1,1,10,1499,149,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
912995,2017-12-27,10,50,63,2221,511,60.0,2173.0,474.0
912996,2017-12-28,10,50,59,2429,587,43.0,2128.0,469.0
912997,2017-12-29,10,50,74,2687,596,68.0,2432.0,566.0
912998,2017-12-30,10,50,62,2742,612,63.0,2484.0,585.0


In [233]:
train_df = df.loc[df['date'] < '2017-01-01'].copy()
test_df = df.loc[(df['date'] >= '2017-01-01') & (df['store'] == 5) &
           (df['item'] == 44)].copy()

In [234]:
train_df

,date,store,item,sales,store_sales,item_sales,sales_lag_365_days,store_sales_lag_365_days,item_sales_lag_365_days
0,2013-01-01,1,1,13,1316,133,NaN,NaN,NaN
1,2013-01-02,1,1,11,1264,99,NaN,NaN,NaN
2,2013-01-03,1,1,14,1305,127,NaN,NaN,NaN
3,2013-01-04,1,1,13,1452,145,NaN,NaN,NaN
4,2013-01-05,1,1,10,1499,149,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
912630,2016-12-27,10,50,60,2173,474,42.0,1820.0,389.0
912631,2016-12-28,10,50,43,2128,469,45.0,2040.0,476.0
912632,2016-12-29,10,50,68,2432,566,51.0,2074.0,456.0
912633,2016-12-30,10,50,63,2484,585,44.0,2187.0,473.0


In [263]:
test_df.sample(5)

,date,store,item,sales,store_sales,item_sales,sales_lag_365_days,store_sales_lag_365_days,item_sales_lag_365_days,dayofweek,quarter,month,dayofyear,dayofmonth,weekofyear,season
794046,2017-04-12,5,44,27,2165,330,18.0,2139.0,304.0,2,2,4,102,12,15,1
794085,2017-05-21,5,44,37,3051,446,23.0,2711.0,378.0,6,2,5,141,21,20,1
794301,2017-12-23,5,44,16,1832,266,21.0,1684.0,252.0,5,4,12,357,23,51,0
793983,2017-02-08,5,44,14,1538,192,27.0,1471.0,242.0,2,1,2,39,8,6,0
794057,2017-04-23,5,44,30,2844,399,25.0,2509.0,395.0,6,2,4,113,23,16,1


In [264]:
def create_date_time_features(data):
    """
    Создание тайм-сириес фичей
    """
    data = data.copy(deep=True)
    data['dayofweek'] = data.date.dt.dayofweek
    data['quarter'] = data.date.dt.quarter
    data['month'] = data.date.dt.month
    data['dayofyear'] = data.date.dt.dayofyear
    data['dayofmonth'] = data.date.dt.day
    data['weekofyear'] = data.date.dt.isocalendar().week.astype("int64")
    # 0: Зима
    # 1: Весна
    # 2: Лето
    # 3: Осень
    data["season"] = np.where(data.month.isin([12,1,2]), 0, 1)
    data["season"] = np.where(data.month.isin([6,7,8]), 2, data["season"])
    data["season"] = np.where(data.month.isin([9, 10, 11]), 3, data["season"])
    return data

In [265]:
train_df = create_date_time_features(train_df)
test_df = create_date_time_features(test_df)

In [266]:
target_pair

(5, 44)

In [267]:
train_df[(train_df['store'] == 5) & (train_df['item'] == 44)]

,date,store,item,sales,store_sales,item_sales,sales_lag_365_days,store_sales_lag_365_days,item_sales_lag_365_days,dayofweek,quarter,month,dayofyear,dayofmonth,weekofyear,season
792484,2013-01-01,5,44,13,1032,154,NaN,NaN,NaN,1,1,1,1,1,1,0
792485,2013-01-02,5,44,8,997,165,NaN,NaN,NaN,2,1,1,2,2,1,0
792486,2013-01-03,5,44,9,1130,170,NaN,NaN,NaN,3,1,1,3,3,1,0
792487,2013-01-04,5,44,16,1258,189,NaN,NaN,NaN,4,1,1,4,4,1,0
792488,2013-01-05,5,44,17,1154,202,NaN,NaN,NaN,5,1,1,5,5,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
793940,2016-12-27,5,44,16,1467,215,14.0,1218.0,182.0,1,4,12,362,27,52,0
793941,2016-12-28,5,44,20,1473,210,12.0,1381.0,221.0,2,4,12,363,28,52,0
793942,2016-12-29,5,44,15,1574,232,16.0,1423.0,166.0,3,4,12,364,29,52,0
793943,2016-12-30,5,44,23,1765,268,19.0,1511.0,225.0,4,4,12,365,30,52,0


In [268]:
target_shop_train_df = train_df[(train_df['store'] == 5) & (train_df['item'] == 44)].copy(deep=True)

## Создадим наши датасеты для обучения и предсказаний

In [269]:
target_shop_train_df.shape, test_df.shape

((1461, 16), (365, 16))

In [270]:
final_train_df = pd.get_dummies(target_shop_train_df, columns=['dayofweek', "quarter", 'month', "season"])

In [271]:
y_train = final_train_df[final_train_df['sales_lag_365_days'].notna()]['sales']
X_train = final_train_df[final_train_df['sales_lag_365_days'].notna()].drop(['date', 'sales', 'store_sales',
                                                                             'item_sales', 'item', 'store'],
                                                                            axis=1)

In [272]:
final_test_df = pd.get_dummies(test_df, columns=['dayofweek', "quarter", 'month', "season"])
y_test = final_test_df[final_test_df['sales_lag_365_days'].notna()]['sales']
X_test = final_test_df[final_test_df['sales_lag_365_days'].notna()].drop(['date', 'sales', 'store_sales',
                                                                             'item_sales', 'item', 'store'],
                                                                            axis=1)

In [273]:
print("X dataframes' shapes:", X_train.shape, X_test.shape)

X dataframes' shapes: (1096, 33) (365, 33)


In [274]:
print("y dataframes' shapes:", y_train.shape, y_test.shape)

y dataframes' shapes: (1096,) (365,)


# Графики перед предсказаниями

In [281]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=pd.date_range("2014-01-01", "2016-12-31", freq='D'),
                         y=y_train,
                         name='Обучающая<br>выборка',
                         mode='lines'))

fig.add_trace(go.Scatter(x=pd.date_range("2017-01-01", "2017-12-31", freq='D'),
                         y=y_test,
                         name='Тестовая<br>выборка',
                         mode='lines'))

fig.update_layout(title='Продажи<br>'
                        '<b>Обучающая выборка</b> vs <b>Тестовая выборка</b>')
fig.update_xaxes(title='Дата', tickformat="%d %b %Y")
fig.update_yaxes(title='Продажи за день')

fig.show()

In [283]:
pd.concat([X_train, y_train], ignore_index=True, axis=1)

,0,1,2,3,4,5,6,7,8,9,...,24,25,26,27,28,29,30,31,32,33
792849,13.0,1032.0,154.0,1,1,1,0,0,1,0,...,0,0,0,0,0,1,0,0,0,15
792850,8.0,997.0,165.0,2,2,1,0,0,0,1,...,0,0,0,0,0,1,0,0,0,15
792851,9.0,1130.0,170.0,3,3,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,8
792852,16.0,1258.0,189.0,4,4,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,17
792853,17.0,1154.0,202.0,5,5,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
793940,14.0,1218.0,182.0,362,27,52,0,1,0,0,...,0,0,0,0,1,1,0,0,0,16
793941,12.0,1381.0,221.0,363,28,52,0,0,1,0,...,0,0,0,0,1,1,0,0,0,20
793942,16.0,1423.0,166.0,364,29,52,0,0,0,1,...,0,0,0,0,1,1,0,0,0,15
793943,19.0,1511.0,225.0,365,30,52,0,0,0,0,...,0,0,0,0,1,1,0,0,0,23


In [289]:
pd.DataFrame(pd.concat([X_train, y_train], ignore_index=True, axis=1),
             columns=X_train.columns.tolist() +
                     ['target']).to_parquet('data/model_datasets/train.parquet')
pd.DataFrame(pd.concat([X_test, y_test], ignore_index=True, axis=1),
             columns=X_test.columns.tolist() +
                     ['target']).to_parquet('data/model_datasets/test.parquet')